In [1]:
import numpy as np
import sklearn as sk
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from sqlite_api import *

In [46]:
translateTeams = {'Arizona Cardinals' : 'ARZ',
                'Atlanta Falcons' : 'ATL',
                'Baltimore Ravens': 'BAL',
                'Buffalo Bills': 'BUF',
                'Carolina Panthers': 'CAR',
                'Chicago Bears': 'CHI',
                'Cincinnati Bengals': 'CIN',
                'Cleveland Browns': 'CLE',
                'Dallas Cowboys': 'DAL',
                'Denver Broncos': 'DEN',
                'Detroit Lions': 'DET',
                'Green Bay Packers': 'GB',
                'Houston Texans': 'HOU',
                'Indianapolis Colts': 'IND',
                'Jacksonville Jaguars': 'JAX',
                'Kansas City Chiefs': 'KC',
                'Miami Dolphins': 'MIA',
                'Minnesota Vikings': 'MIN',
                'New England Patriots': 'NE',
                'New Orleans Saints': 'NO',
                'New York Giants': 'NYG',
                'New York Jets': 'NYJ',
                'Oakland Raiders': 'OAK',
                'Philadelphia Eagles': 'PHI',
                'Pittsburgh Steelers': 'PIT',
                'Los Angeles Chargers': 'LAC',
                'Seattle Seahawks': 'SEA',
                'San Francisco 49ers': 'SF',
                'Los Angeles Rams': 'LAR',
                'Tampa Bay Buccaneers': 'TB',
                'Tennessee Titans': 'TEN',
                'Washington Redskins': 'WAS'}

In [55]:
def scrape(translate):
    # column name for the dataframe to be created
    labels = ['Player', 'Team', 'Plays', 'FPT', 'Games', 'FPG', 'RushAttempts', 
              'RushYrds', 'RushTDs', 'PassAttempts', 'Complete', 'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions']
    
    # build pandas dataframe for this season's data
    df = pd.DataFrame(columns=labels)
    
    # build url and request html
    url = 'http://www.cbssports.com/nfl/news/2017-nfl-schedule-release-heres-the-time-and-date-for-all-256-games/'
    r  = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    contents = BeautifulSoup(r.text, 'lxml')
    
    # hard-coded number of columns in table
    num_cols = 15
    table = contents.find_all("td", class_="xl66")
    entries = contents.find_all("td", {'class':['xl72', 'xl66']})
    
    df = pd.DataFrame(columns=['Home', 'Away', 'Week'])
    
    cur_week = 0
    for row in entries:
        string = row.text.strip()
        if 'WEEK' in string:
            cur_week += 1
        elif 'at' in string:
            string = string.replace(' (London)', '')
            string = string.replace(' (Mexico City)', '')
            string = string.replace(' *', '')
            teamStr = string.split(' at ')
            df = df.append({'Home': translateTeams[teamStr[1].strip()], 'Away': translateTeams[teamStr[0].strip()], 
                            'Week': cur_week}, ignore_index=True)
            
    return df
    
sched = scrape(translateTeams)

In [57]:
sched.to_json('SCHEDULE2017.json', orient='index')